In [2]:
from typing import List
from datetime import datetime, timedelta
import secrets

from sqlalchemy.orm import Session

from sqlalchemy import asc, or_, and_, func, not_
from sqlalchemy.sql import text

from sql_app import crud, models, schemas
from sql_app.database import SessionLocal, engine

models.Base.metadata.create_all(bind=engine)

db = SessionLocal()

date_from = datetime.now()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).join(models.Reservation,isouter=True).\
        filter(or_(or_(date_to < models.Reservation.pick_up_date, date_from > models.Reservation.return_date),\
               models.CostumeItem.reservation_id == None))
    all_costumes = db.query(models.CostumeItem)
    
finally:
    db.close()


In [3]:
#costumes = schemas.CostumeItem(**available_costumes.__dict__)
for c in available_costumes.all():
    print(c.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab15b0>, 'reservation_id': 11, 'location_id': 1, 'id': 3, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab17f0>, 'reservation_id': 11, 'location_id': 1, 'id': 2, 'rental_id': None, 'model_id': 2}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1820>, 'reservation_id': 12, 'location_id': 1, 'id': 7, 'rental_id': None, 'model_id': 7}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1880>, 'reservation_id': 15, 'location_id': 1, 'id': 1, 'rental_id': None, 'model_id': 1}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab18e0>, 'reservation_id': None, 'location_id': 2, 'id': 8, 'rental_id': None, 'model_id': 3}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1940>, 'reservation_id': None, 'location_id': 1, 'id': 6, 'rental_id': Non

In [4]:
for c in all_costumes.all():
    print(c.__dict__)

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1a00>, 'reservation_id': None, 'location_id': 2, 'id': 4, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1880>, 'reservation_id': None, 'location_id': 2, 'id': 5, 'rental_id': None, 'model_id': 4}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1970>, 'reservation_id': None, 'location_id': 1, 'id': 6, 'rental_id': None, 'model_id': 6}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1670>, 'reservation_id': None, 'location_id': 2, 'id': 8, 'rental_id': None, 'model_id': 3}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1700>, 'reservation_id': 11, 'location_id': 1, 'id': 2, 'rental_id': None, 'model_id': 2}
{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState object at 0x7f0220ab1760>, 'reservation_id': 11, 'location_id': 1, 'id': 3, 'rental_id':

In [5]:
len(available_costumes.all())

8

# delete all reservations and rentals

In [9]:
db = SessionLocal()

date_from = datetime.now()
date_to = date_from+timedelta(days=10)
try:
    available_costumes = db.query(models.CostumeItem).join(models.Reservation).\
        filter(or_(date_to < models.Reservation.pick_up_date, date_from > models.Reservation.return_date))
    items = db.query(models.CostumeItem).all()
    #db.query(models.Reservation).delete()
    for item in items:
        item.reservation_id = None
        item.rental_id = None
    db.commit()
    
    db.query(models.CostumeRental).delete()
    db.query(models.Reservation).delete()
    
    db.commit()
finally:
    db.close()